<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/rapids_cudf_pandas_accelerator_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10 Minutes to RAPIDS cuDF's pandas accelerator mode (cudf.pandas)

cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API in the style of pandas.

cuDF now provides a pandas accelerator mode (`cudf.pandas`), allowing you to bring accelerated computing to your pandas workflows without requiring any code change.

This notebook is a short introduction to `cudf.pandas`.

# ⚠️ Verify your setup

First, we'll verify that you are running with an NVIDIA GPU.

In [1]:
!nvidia-smi  # this should display information about available GPUs

Sun Dec 24 21:04:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

With our GPU-enabled Colab runtime active, we'll now install cuDF.

If you're interested in installing on other platforms, please visit https://rapids.ai/#quick-start to learn more.

In [2]:
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      S

In [3]:
import cudf  # this should work without any errors

We'll also install `plotly-express` for visualizing data.

### Environment Note
If you're not running this notebook on Colab, you may need to reload the webpage for the `plotly.express` visualizations to work correctly.


In [4]:
!pip install plotly-express

# Download the data

The data we'll be working with is the [Parking Violations Issued - Fiscal Year 2022](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/7mxj-7a6y) dataset from NYC Open Data.

We're downloading a copy of this dataset from an s3 bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

## Data License and Terms
As this dataset originates from the NYC Open Data Portal, it's governed by their license and terms of use.

### Are there restrictions on how I can use Open Data?

> Open Data belongs to all New Yorkers. There are no restrictions on the use of Open Data. Refer to Terms of Use for more information.

### [Terms of Use](https://opendata.cityofnewyork.us/overview/#termsofuse)

> By accessing datasets and feeds available through NYC Open Data, the user agrees to all of the Terms of Use of NYC.gov as well as the Privacy Policy for NYC.gov. The user also agrees to any additional terms of use defined by the agencies, bureaus, and offices providing data. Public data sets made available on NYC Open Data are provided for informational purposes. The City does not warranty the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set made available on NYC Open Data, nor are any such warranties to be implied or inferred with respect to the public data sets furnished therein.

> The City is not liable for any deficiencies in the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set, or application utilizing such data set, provided by any third party.

> Submitting City Agencies are the authoritative source of data available on NYC Open Data. These entities are responsible for data quality and retain version control of data sets and feeds accessed on the Site. Data may be updated, corrected, or refreshed at any time.

In [5]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2023-12-24 21:06:04--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 13.224.250.77, 13.224.250.33, 13.224.250.96, ...
Connecting to data.rapids.ai (data.rapids.ai)|13.224.250.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477161608 (455M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 455.06M  17.9MB/s    in 27s     

2023-12-24 21:06:32 (17.0 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [477161608/477161608]



# Analysis using Standard Pandas

First, let's use Pandas to read in some columns of the dataset:

In [6]:
import pandas as pd

In [7]:
# read 5 columns data:
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# view a random sample of 10 rows:
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
13395583,NY,FAILURE TO STOP AT RED LIGHT,SUBN,04/30/2022,5120550216
9264892,NY,40-Fire Hydrant,4DSD,02/05/2022,8876711880
3911863,NY,74-Missing Display Plate,SUBN,09/22/2021,8962576405
2709201,NY,38-Failure to Dsplay Meter Rec,SUBN,08/19/2021,8974124634
5244098,NY,71A-Insp Sticker Expired (NYS),SUBN,10/01/2021,8962113132
14249796,NY,71A-Insp Sticker Expired (NYS),4DSD,05/27/2022,8571234164
1501202,NY,PHTO SCHOOL ZN SPEED VIOLATION,2DSD,08/10/2021,4743997227
15109325,NY,69-Fail to Dsp Prking Mtr Rcpt,VAN,06/14/2022,8885956452
725304,IL,21-No Parking (street clean),4DSD,07/02/2021,8945208033
9655541,NY,38-Failure to Dsplay Meter Rec,SUBN,02/08/2022,8813729649


Next, we'll try to answer a few questions using the data.

## Which parking violation is most commonly committed by vehicles from various U.S states?

Each record in our dataset contains the state of registration of the offending vehicle, and the type of parking offence. Let's say we want to get the most common type of offence for vehicles registered in different states. We can do this in Pandas using a combination of [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) and [GroupBy.head](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.head.html):

In [ ]:
(df[["Registration State", "Violation Description"]]  # get only these two columns
 .value_counts()  # get the count of offences per state and per type of offence
 .groupby("Registration State")  # group by state
 .head(1)  # get the first row in each group (the type of offence with the largest count)
 .sort_index()  # sort by state name
 .reset_index()
)

,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


The code above uses [method chaining](https://tomaugspurger.net/posts/method-chaining/) to combine a series of operations into a single statement. You might find it useful to break the code up into multiple statements and inspect each of the intermediate results!

## Which vehicle body types are most frequently involved in parking violations?

We can also investigate which vehicle body types most commonly appear in parking violations

In [ ]:
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


## How do parking violations vary across days of the week?

In [ ]:
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

It looks like there are fewer violations on weekends, which makes sense! During the week, more people are driving in New York City.

## Let's time it!

Loading and processing this data took a little time. Let's measure how long these pipelines take in Pandas:

In [ ]:
%%time

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 11.9 s, sys: 3.25 s, total: 15.2 s
Wall time: 13.6 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [ ]:
%%time

(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 2.53 s, sys: 189 ms, total: 2.72 s
Wall time: 2.7 s


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


In [ ]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 8.55 s, sys: 729 ms, total: 9.28 s
Wall time: 9.48 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

# Using cudf.pandas

Now, let's re-run the Pandas code above with the `cudf.pandas` extension loaded.

Typically, you should load the `cudf.pandas` extension as the first step in your notebook, before importing any modules. Here, we explicitly restart the kernel to simulate that behavior.

In [8]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
%load_ext cudf.pandas

In [2]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 601 ms, sys: 309 ms, total: 909 ms
Wall time: 1.2 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [3]:
%%time

(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 19.9 ms, sys: 15.6 ms, total: 35.5 ms
Wall time: 71.7 ms


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
YANT,1
YBSD,1
YEL,1


In [4]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 230 ms, sys: 67.7 ms, total: 297 ms
Wall time: 285 ms


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

Much faster! Operations that took 5-20 seconds can now potentially finish in just milliseconds without changing any code.

# Understanding Performance

`cudf.pandas` provides profiling utilities to help you better understand performance. With these tools, you can identify which parts of your code ran on the GPU and which parts ran on the CPU.

They're accessible in the `cudf.pandas` namespace since the `cudf.pandas` extension was loaded above with `load_ext cudf.pandas`.

#### Colab Note
If you're running in Colab, the first time you run use the profiler it may take 10+ seconds due to Colab's debugger interacting with the built-in Python function [sys.settrace](https://docs.python.org/3/library/sys.html#sys.settrace) that we use for profiling. For demo purposes, this isn't an issue. Just run the cell again.

## Profiling Functionality

We can generate a per-function profile:

In [5]:
%%cudf.pandas.profile

small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 97, in __enter__
    sys.settrace(self._tracefunc)

/usr/local/lib/python3.10/dist-packages/cudf/core/dataframe.py:5933: UserWarning: Row-wise operations currently only support int, float and bool dtypes. Non numeric columns are ignored.
  warnings.warn(msg)

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr

                                                                                                         
                                       Total time elapsed: 12.210 seconds                                
                                     5 GPU function calls in 11.694 seconds                              
                                      1 CPU function calls in 0.022 seconds                              
                                                                                                         
                                                      Stats                                              
                                                                                                         
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function            ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame           │ 1          │ 0.004       │ 0.004       │ 0          │ 0.000       │ 0.000       │
│ concat              │ 1          │ 0.014       │ 0.014       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.min       │ 1          │ 11.666      │ 11.666      │ 1          │ 0.022       │ 0.022       │
│ DataFrame.groupby   │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ SeriesGroupBy.count │ 1          │ 0.010       │ 0.010       │ 0          │ 0.000       │ 0.000       │
└─────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame.min

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=88503;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

In [6]:
%%cudf.pandas.line_profile

small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()

/usr/local/lib/python3.10/dist-packages/cudf/core/dataframe.py:5933: UserWarning: Row-wise operations currently only support int, float and bool dtypes. Non numeric columns are ignored.
  warnings.warn(msg)


                                                                                                              
                                          Total time elapsed: 4.114 seconds                                   
                                                                                                              
                                                        Stats                                                 
                                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]}) │ 0.003728868 │             │
│          │                                                                     │             │             │
│ 3        │     small_df = pd.concat([small_df, small_df])                      │ 0.025658616 │             │
│          │                                                                     │             │             │
│ 5        │     axis = 0                                                        │             │             │
│          │                                                                     │             │             │
│ 6        │     for i in range(0, 2):                                           │             │             │
│          │                                                                     │             │             │
│ 7        │         small_df.min(axis=axis)                                     │ 0.050977370 │ 0.036330143 │
│          │                                                                     │             │             │
│ 8        │         axis = 1                                                    │             │             │
│          │                                                                     │             │             │
│ 10       │     counts = small_df.groupby("a").b.count()                        │ 0.020166352 │             │
│          │                                                                     │             │             │
└──────────┴─────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

## Behind the scenes: What's going on here?

When you load `cudf.pandas`, Pandas types like `Series` and `DataFrame` are replaced by proxy objects that dispatch operations to cuDF when possible. We can verify that `cudf.pandas` is active by looking at our `pd` variable:

In [7]:
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

As a result, all pandas functions, methods, and created objects are proxies:

In [8]:
type(pd.read_csv)

cudf.pandas.fast_slow_proxy._FunctionProxy

Operations supported by cuDF will be **very** fast:

In [9]:
%%time
df.count(axis=0)

CPU times: user 2.44 ms, sys: 1.13 ms, total: 3.57 ms
Wall time: 3.98 ms


Registration State       15435607
Violation Description    15117819
Vehicle Body Type        15402365
Issue Date               15435607
Summons Number           15435607
issue_weekday            15435607
dtype: int64

Operations not supported by cuDF will be slower, as they fall back to using Pandas (copying data between the CPU and GPU under the hood as needed). For example, cuDF does not currently support the `axis=` parameter to the `count` method. So this operation will run on the CPU and be noticeably slower than the previous one.

In [10]:
%%time
df.count(axis=1) # This will use pandas, because cuDF doesn't support axis=1 for the .count() method

CPU times: user 15.4 s, sys: 2.64 s, total: 18 s
Wall time: 17.6 s


0           5
1           5
2           5
3           5
4           5
           ..
15435602    6
15435603    6
15435604    6
15435605    6
15435606    6
Length: 15435607, dtype: int64

But the story doesn't end here. We often need to mix our own code with third-party libraries that other people have written. Many of these libraries accept pandas objects as inputs.

# Using third-party libraries with cudf.pandas

You can pass Pandas objects to third-party libraries when using `cudf.pandas`, just like you would when using regular Pandas.

Below, we show an example of using [plotly-express](https://plotly.com/python/plotly-express/) to visualize the data we've been processing:

## Visualizing which states have more pickup trucks relative to other vehicles?

In [11]:
import plotly.express as px

df = df.rename(columns={
    "Registration State": "reg_state",
    "Vehicle Body Type": "vehicle_type",
})

# vehicle counts per state:
counts = df.groupby("reg_state").size().sort_index()
# vehicles with type "PICK" (Pickup Truck)
pickup_counts = df.where(df["vehicle_type"] == "PICK").groupby("reg_state").size()
# percentage of pickup trucks by state:
pickup_frac = ((pickup_counts / counts) * 100).rename("% Pickup Trucks")
del pickup_frac["MB"]  # (Manitoba is a huge outlier!)

# plot the results:
pickup_frac = pickup_frac.reset_index()
px.choropleth(pickup_frac, locations="reg_state", color="% Pickup Trucks", locationmode="USA-states", scope="usa")

## Beyond just passing data: **Accelerating** third-party code

Being able to pass these proxy objects to libraries like Plotly is great, but the benefits don't end there.

When you enable `cudf.pandas`, pandas operations running **inside the third-party library's functions** will also benefit from GPU acceleration where possible!

Below, you can see an image illustrating how `cudf.pandas` can accelerate the pandas backend in Ibis, a library that provides a unified DataFrame API to various backends. We ran this example on a system with an NVIDIA H100 GPU and an Intel Xeon Platinum 8480CL CPU.


By loading the `cudf.pandas` extension, pandas operations within Ibis can use the GPU with zero code change. It just works.

![ibis](https://drive.google.com/uc?id=1uOJq2JtbgVb7tb8qw8a2gG3JRBo72t_H)

# Conclusion

With `cudf.pandas`, you can keep using pandas as your primary dataframe library. When things start to get a little slow, just load the `cudf.pandas` and run your existing code on a GPU!

To learn more, we encourage you to visit [rapids.ai/cudf-pandas](https://rapids.ai/cudf-pandas).